# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [57]:
import pandas as pd
import os 

schools_filepath = os.path.join("raw_data","schools_complete.csv")
students_filepath = os.path.join("raw_data","students_complete.csv")

df_schools = pd.read_csv(schools_filepath)
df_students = pd.read_csv(students_filepath)

df_students.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [58]:
df_schools.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


## District Summary

In [59]:
#Creating a high level snapshot (in a table format) of the district's key metrics

schools_total = len(df_schools)
students_total = df_schools["size"].sum()
total_budget = df_schools["budget"].sum()
avg_math_score = df_students["math_score"].mean()
avg_reading_score = df_students["reading_score"].mean()

cutoff_score_math = df_students.loc[df_students["math_score"] >= 70,:]
students_pass_math = len(cutoff_score_math)
percentage_pass_score_math = (students_pass_math / students_total)*100

cutoff_score_reading = df_students.loc[df_students["reading_score"] >= 70,:]
students_pass_reading = len(cutoff_score_reading)
percentage_pass_score_reading = (students_pass_reading/students_total)*100

pass_rate = (percentage_pass_score_math + percentage_pass_score_reading)/2

# Format items
total_budget = '${:,.0f}'.format(total_budget)
avg_math_score = '{0:.2f}'.format(avg_math_score)
avg_reading_score = '{0:.2f}'.format(avg_reading_score)
percentage_pass_score_math = '{0:.2f}%'.format(percentage_pass_score_math)
percentage_pass_score_reading = '{0:.2f}%'.format(percentage_pass_score_reading)
pass_rate = '{0:.2f}%'.format(pass_rate)


# Summary Table
district_summary = pd.DataFrame({
    "Total Schools":[schools_total],
    "Total Students":[students_total],
    "Total Budget":[total_budget],
    "Average Math Score":[avg_math_score],
    "Average Reading Score":[avg_reading_score],
    "% Passing Math":[percentage_pass_score_math],
    "% Passing Reading":[percentage_pass_score_reading],
    "Overall Passing Rate":[pass_rate]
})

# Reset column order
district_summary = district_summary[["Total Schools",
                                     "Total Students",
                                     "Total Budget",
                                     "Average Math Score",
                                     "Average Reading Score",
                                     "% Passing Math",
                                     "% Passing Reading",
                                     "Overall Passing Rate"]]

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,"$24,649,428",78.99,81.88,74.98%,85.81%,80.39%


## School Summary

In [60]:
#Create an overview table that summarizes key metrics about each school.

student_in_school = df_students.groupby("school")

df_schools["Per Student Budget"] = df_schools["budget"] / df_schools["size"]

school_performance = pd.DataFrame({})
school_performance["Average Math Score"] = student_in_school["math_score"].mean()
school_performance["Average Reading Score"] = student_in_school["reading_score"].mean()

#student_passed_math,,,,passed_math_by_school
student_passed_math = cutoff_score_math.groupby("school")
student_passed_math_counts = student_passed_math.count()
school_performance["Number Passing Math"] = student_passed_math_counts["name"]


student_passed_reading = cutoff_score_reading.groupby("school")
student_passed_reading_counts = student_passed_reading.count()
school_performance["Number Passing Reading"] = student_passed_reading_counts["name"]


school_performance = school_performance.reset_index()
school_performance = school_performance.rename(columns={"school":"name"})

summary_on_school = pd.merge(df_schools, school_performance, on="name")
summary_on_school["% Passing Math"] = (summary_on_school["Number Passing Math"] / summary_on_school["size"])*100
summary_on_school["% Passing Reading"] = (summary_on_school["Number Passing Reading"] / summary_on_school["size"])*100
summary_on_school["% Overall Passing Rate"] = (summary_on_school["% Passing Math"] + summary_on_school["% Passing Reading"])/2


# Summary table
school_overview = pd.DataFrame({
    "School Name":summary_on_school["name"],
    "School Type":summary_on_school["type"],
    "Total Students":summary_on_school["size"],
    "Total Budget":summary_on_school["budget"],
    "Per Student Budget":summary_on_school["Per Student Budget"],
    "Average Math Score":summary_on_school["Average Math Score"],
    "Average Reading Score":summary_on_school["Average Reading Score"],
    "% Passing Math":summary_on_school["% Passing Math"],
    "% Passing Reading":summary_on_school["% Passing Reading"],
    "% Overall Passing Rate":summary_on_school["% Overall Passing Rate"] 
    })

school_overview = school_overview.set_index("School Name")

# Format items
school_overview["% Passing Math"] = school_overview["% Passing Math"].map('{0:.2f}%'.format)
school_overview["% Passing Reading"] = school_overview["% Passing Reading"].map('{0:.2f}%'.format)
school_overview["Average Math Score"] = school_overview["Average Math Score"].map('{0:.2f}'.format)
school_overview["Average Reading Score"] = school_overview["Average Reading Score"].map('{0:.2f}'.format)
school_overview["% Overall Passing Rate"] = school_overview["% Overall Passing Rate"].map('{0:.2f}%'.format)
school_overview["Per Student Budget"] = school_overview["Per Student Budget"].map('${:,.2f}'.format)
school_overview["Total Budget"] = school_overview["Total Budget"].map('${:,.0f}'.format)

# Reset column order
school_overview= school_overview[["School Type",
                                  "Total Students",
                                  "Total Budget",
                                  "Per Student Budget",
                                  "Average Math Score",
                                  "Average Reading Score",
                                  "% Passing Math",
                                  "% Passing Reading",
                                  "% Overall Passing Rate"]]
school_overview.head()


,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,"$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Figueroa High School,District,2949,"$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Shelton High School,Charter,1761,"$1,056,600",$600.00,83.36,83.73,93.87%,95.85%,94.86%
Hernandez High School,District,4635,"$3,022,020",$652.00,77.29,80.93,66.75%,80.86%,73.81%
Griffin High School,Charter,1468,"$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.27%


## Top Performing Schools (By Passing Rate)

In [61]:
# Create a table that highlights the top 5 performing schools based on Overall Passing Rate
top_performing_schools = school_overview.sort_values("% Overall Passing Rate", ascending=False)
top_performing_schools = top_performing_schools.head(5)
top_performing_schools

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,"$1,043,130",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,Charter,1468,"$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,Charter,2283,"$1,319,574",$578.00,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

In [62]:
# Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate
bottom_performing_schools = school_overview.sort_values("% Overall Passing Rate", ascending=True)
bottom_performing_schools = bottom_performing_schools.head(5)
bottom_performing_schools

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,2949,"$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,2917,"$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,4761,"$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,2739,"$1,763,916",$644.00,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

In [63]:
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

ninth_grade = df_students.loc[df_students["grade"] == "9th",:]
tenth_grade = df_students.loc[df_students["grade"] == "10th",:]
eleventh_grade = df_students.loc[df_students["grade"] == "11th",:]
twelveth_grade = df_students.loc[df_students["grade"] == "12th",:]

math_grade = pd.DataFrame({}) 

ninth_grade_class = ninth_grade.groupby("school")
nineth_grade_math = ninth_grade_class["math_score"].mean()
math_grade["Ninth"] = nineth_grade_math

tenth_grade_class = tenth_grade.groupby("school")
tenth_grade_math = tenth_grade_class["math_score"].mean()
math_grade["Tenth"] = tenth_grade_math

eleventh_grade_class = eleventh_grade.groupby("school")
eleventh_grade_math = eleventh_grade_class["math_score"].mean()
math_grade["Eleventh"] = eleventh_grade_math

twelveth_grade_class = twelveth_grade.groupby("school")
twelveth_grade_math = twelveth_grade_class["math_score"].mean()
math_grade["Twelveth"] = twelveth_grade_math

math_grade

,Ninth,Tenth,Eleventh,Twelveth
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [64]:
# Create a table that lists the average Reading Score for students of 
#each grade level (9th, 10th, 11th, 12th) at each school

reading_grade = pd.DataFrame({}) 

ninth_grade_reading = ninth_grade_class["reading_score"].mean()
reading_grade["Ninth"] = ninth_grade_reading

tenth_grade_reading = tenth_grade_class["reading_score"].mean()
reading_grade["Tenth"] = tenth_grade_reading

eleventh_grade_reading = eleventh_grade_class["reading_score"].mean()
reading_grade["Eleventh"] = eleventh_grade_reading

twelveth_grade_reading = twelveth_grade_class["reading_score"].mean()
reading_grade["Twelveth"] = twelveth_grade_reading

reading_grade

,Ninth,Tenth,Eleventh,Twelveth
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [65]:
# Create a table that breaks down school performances based on average Spending Ranges (Per Student)

summary_on_school["Per Student Budget"] = summary_on_school["Per Student Budget"].map(int)

# Set base sizes and groups
containers = [0,585,615,645,1000]
groups = ["<$585","$585-615","$615-645","$645-675"]
summary_on_school["Spending Range (Per Student)"] = pd.cut(summary_on_school["Per Student Budget"], containers, labels=groups)

school_spending = summary_on_school.groupby("Spending Range (Per Student)")
summary_school_spending = pd.DataFrame({}) 

# Build summary table
summary_school_spending["Average Math Score"] = school_spending["Average Math Score"].mean()
summary_school_spending["Average Reading Score"] = school_spending["Average Reading Score"].mean()
summary_school_spending["% Passing Math"] = school_spending["% Passing Math"].mean()
summary_school_spending["% Passing Reading"] = school_spending["% Passing Reading"].mean()
summary_school_spending["% Overall Passing Rate"] = school_spending["% Overall Passing Rate"].mean()

# Format summary table
summary_school_spending["% Passing Math"] = summary_school_spending["% Passing Math"].map('{0:.2f}%'.format)
summary_school_spending["% Passing Reading"] = summary_school_spending["% Passing Reading"].map('{0:.2f}%'.format)
summary_school_spending["Average Math Score"] = summary_school_spending["Average Math Score"].map('{0:.2f}'.format)
summary_school_spending["Average Reading Score"] = summary_school_spending["Average Reading Score"].map('{0:.2f}'.format)
summary_school_spending["% Overall Passing Rate"] = summary_school_spending["% Overall Passing Rate"].map('{0:.2f}%'.format)
summary_school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Range (Per Student),,,,,
<$585,83.46,83.93,93.46%,96.61%,95.04%
$585-615,83.60,83.89,94.23%,95.90%,95.07%
$615-645,79.08,81.89,75.67%,86.11%,80.89%
$645-675,77.00,81.03,66.16%,81.13%,73.65%


## Scores by School Size

In [66]:
# Time group schools based on a reasonable approximation of school size

summary_on_school["size"] = summary_on_school["size"].map(int)

# Set base sizes and groups
base = [0, 1750, 3500, 5000]
groups = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]
summary_on_school["School Size"] = pd.cut(summary_on_school["size"], base, labels=groups)

school_size = summary_on_school.groupby("School Size")
summary_school_size = pd.DataFrame({}) 

# Build summary table
summary_school_size["Average Math Score"] = school_size["Average Math Score"].mean()
summary_school_size["Average Reading Score"] = school_size["Average Reading Score"].mean()
summary_school_size["% Passing Math"] = school_size["% Passing Math"].mean()
summary_school_size["% Passing Reading"] = school_size["% Passing Reading"].mean()
summary_school_size["% Overall Passing Rate"] = school_size["% Overall Passing Rate"].mean()

# Format summary table
summary_school_size["% Passing Math"] = summary_school_size["% Passing Math"].map('{0:.2f}%'.format)
summary_school_size["% Passing Reading"] = summary_school_size["% Passing Reading"].map('{0:.2f}%'.format)
summary_school_size["Average Math Score"] = summary_school_size["Average Math Score"].map('{0:.2f}'.format)
summary_school_size["Average Reading Score"] = summary_school_size["Average Reading Score"].map('{0:.2f}'.format)
summary_school_size["% Overall Passing Rate"] = summary_school_size["% Overall Passing Rate"].map('{0:.2f}%'.format)
summary_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.60,83.88,93.44%,96.66%,95.05%
Medium (1000-2000),80.55,82.68,82.17%,89.63%,85.90%
Large (2000-5000),77.06,80.92,66.46%,81.06%,73.76%


## Scores by School Type

In [68]:
# Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

district_schools = summary_on_school.loc[summary_on_school["type"] == "District", :]
charter_schools = summary_on_school.loc[summary_on_school["type"] == "Charter", :]

# Build summary table
summary_school_type = pd.DataFrame({
    "School Type":["District Schools","Charter Schools"],
    "Average Math Score":[district_schools["Average Math Score"].mean(),charter_schools["Average Math Score"].mean()],
    "Average Reading Score":[district_schools["Average Reading Score"].mean(),charter_schools["Average Reading Score"].mean()],
    "% Passing Math":[district_schools["% Passing Math"].mean(),charter_schools["% Passing Math"].mean()],
    "% Passing Reading":[district_schools["% Passing Reading"].mean(),charter_schools["% Passing Reading"].mean()],
    "% Overall Passing Rate":[district_schools["% Overall Passing Rate"].mean(),charter_schools["% Overall Passing Rate"].mean()]
})

# Set index and order columns
summary_school_type = summary_school_type.set_index("School Type")
summary_school_type = summary_school_type[["Average Math Score",
                                  "Average Reading Score",
                                  "% Passing Math",
                                  "% Passing Reading",
                                  "% Overall Passing Rate"]]

# Format summary table
summary_school_type["% Passing Math"] = summary_school_type["% Passing Math"].map('{0:.2f}%'.format)
summary_school_type["% Passing Reading"] = summary_school_type["% Passing Reading"].map('{0:.2f}%'.format)
summary_school_type["Average Math Score"] = summary_school_type["Average Math Score"].map('{0:.2f}'.format)
summary_school_type["Average Reading Score"] = summary_school_type["Average Reading Score"].map('{0:.2f}'.format)
summary_school_type["% Overall Passing Rate"] = summary_school_type["% Overall Passing Rate"].map('{0:.2f}%'.format)

summary_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
District Schools,76.96,80.97,66.55%,80.80%,73.67%
Charter Schools,83.47,83.90,93.62%,96.59%,95.10%
